In [ ]:
# google drive マウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
cd "/content/drive/My Drive/Colab Notebooks/sprint25_groupwork/dive_groupwork"

/content/drive/My Drive/Colab Notebooks/sprint25_groupwork/dive_groupwork


In [ ]:
!pip install pygame

In [ ]:
#!/usr/bin/env python
#! -*- coding: utf-8 -*-
from keras.preprocessing import image
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import pygame.mixer
import numpy as np
# import picamera
from PIL import Image
from time import sleep, perf_counter

from translate import *
from bottle_master import bottle_master
# from shutter2image import shutter2image
import os
import datetime
import argparse






if __name__ == '__main__':
  
    # 引数を読み込み
    args = parser.parse_args()
    # predictモードの場合
    if args.predict_mode == True and args.model_path is None:
        print('--model_pathを指定してください')
    else:
        #画像を保存する場合、ディレクトリを作成
        if args.image_save:
            try:
                os.makedirs(args.image_save, exist_ok=True)
            except:
                print('読み取った画像を保存するpathが正しく指定されていません')

        # モデル+重みを読込み
        if args.predict_mode is True:
            self_model = load_model(args.model_path)

        #商品価格等の情報をbottle_masterから読み込み
        name_price_dict = bottle_master.bottle_master_dict()
        _, label_dict = bottle_master.bottle_label_dirpath()
        bottle_label = {}
        for key, value in label_dict.items():
            bottle_label[value] = key

        language_dict = {'k':'Kinyarwanda', 'e':'English', 'f':'French', 'j':'Japanese'}
        while True:
            money_sum = 0
            print('which language? Please select a language and input a key!!')
            language = input('Kinyarwanda :「k」,English :「e」,French :「f」,Japanese :「j]')
            if language not in ['k', 'e', 'f', 'j']:
                print('Please input correctly')
                pass
            else:
                print('Language_mode : {}'.format(language_dict[language]))

                while True:
                    # 「商品を置いてenterを押してください」
                    scene = "scan_start"
                    input(translate_language(language, scene))

                    # 時間を図る
                    time_start = perf_counter()

                    # 写真撮影した日時でファイル名を作成
                    dt_now = datetime.datetime.now()
                    file_name = 'checked' + '_' + dt_now.strftime('%Y%m%d%H%M%S') + '.jpg'

                    # 画像をsaveするpathを作成
                    if args.image_save:
                        save_path = os.path.join(args.image_save, file_name)
                    else:
                        save_path = os.path.join(os.getcwd(), file_name)

                    # 写真撮影してjpgをディレクトリへ保管
                    # ローカルデバッグ
                    #shutter2image.shutter(save_path)

                    # # 音声再生
                    # pygame.mixer.music.play(1)
                    # sleep(1)
                    # # 再生の終了
                    # pygame.mixer.music.stop()

                    # 画像をモデルの入力用に加工

                    img = Image.open(save_path)
                    # ローカルデバッグ
                    # img = Image.open('/Users/ikeda/Desktop/IMG_0819.jpg')
                    # 画像保存のpathが指定されていない場合は消去
                    if args.image_save == None:
                        os.remove(save_path)

                    #img = Image.open("./0.jpg")
                    img = img.resize((224, 224))
                    img_array = img_to_array(img)
                    img_array = img_array.astype('float32')/255.0
                    img_array = img_array.reshape((1,224,224,3))

                    # predict_modeによって分岐
                    # modelが読み込まれてなく、流れのみ再現
                    if args.predict_mode == 'False':
                        # 時間計測
                        time_end = perf_counter()
                        # 商品を読み取るまで {}[s] 経過しました
                        scene = "time"
                        print(translate_language(language, scene).format(time_end-time_start))
                        
                        # 商品 : not predict が読み取られました
                        scene = "read_message"
                        print(translate_language(language, scene).format(img_array.shape))
                        
                        print('img_array_shape : {}'.format(img_array.shape))

                        while True:
                            # 読み取られた商品が正しい場合は「y」、誤っていた場合は「n」を押してください
                            scene = "correct?"
                            key = input(translate_language(language, scene))

                            if key != 'y' and key != 'n':
                                # 正しく入力されていません
                                scene = "input_error"
                                print(translate_language(language, scene))
                            else:
                                break

                        if key == 'y':
                            money_sum += 100
                            # 小計
                            scene = "subtotal"
                            print(translate_language(language, scene).format(money_sum))

                            while True:
                                # 続けて商品をスキャンする場合は「y」、会計する場合は「f」、\n キャンセルする商品がある場合は「ｘ」を押して下さい。
                                scene = "continue?"
                                key = input(translate_language(language, scene))
                                if key != 'y' and key != 'f':
                                    # 正しく入力されていません
                                    scene = "input_error"
                                    print(translate_language(language, scene))
                                else:
                                    break
                            if key == 'f':
                                # 合計
                                scene = "total"
                                print(translate_language(language, scene).format(money_sum))
                                
                                # ありがとうございました
                                scene = "thanks"
                                print(translate_language(language, scene))
                                print()
                                break
                            elif key == 'n':
                                # 次の商品を指定の位置に置いてください。
                                scene = "next"
                                print(translate_language(language, scene))


                    # 読み込んだmodelでpridictを実行
                    else:
                    # ここはYOLO合わせで修正
                        img_pred = self_model.predict(img_array)
                        print("debug:",img_pred)
                        name = bottle_label[np.argmax(img_pred)]

                        # 時間計測
                        time_end = perf_counter()
                        # 商品を読み取るまで {}[s] 経過しました
                        scene = "time"
                        print(translate_language(language, scene).format(time_end-time_start))

                        # 商品 : {} 値段 : {} が読み取られました'
                        scene = "read_message"
                        print(translate_language(language, scene).format(name, name_price_dict[name]))

                        while True:
                            # 読み取られた商品が正しい場合は「y」、誤っていた場合は「n」を押してください
                            scene = "correct?"
                            key = input(translate_language(language, scene))
                            if key != 'y' and key != 'n':
                                # 正しく入力されていません
                                scene = "input_error"
                                print(translate_language(language, scene))
                            else:
                                break

                        if key == 'y':
                            money_sum += name_price_dict[name]
                            # 小計
                            scene = "subtotal"
                            print(translate_language(language, scene).format(money_sum))

                            while True:
                                # 続けて商品をスキャンする場合は「y」、会計する場合は「f」、\n キャンセルする商品がある場合は「ｘ」を押して下さい。
                                scene = "continue?"
                                key = input(translate_language(language, scene))
                                if key != 'y' and key != 'f':
                                    # 正しく入力されていません
                                    scene = "input_error"
                                    print(translate_language(language, scene)) 
                                else:
                                    break

                            if key == 'f':
                                # 合計
                                scene = "total"
                                print(translate_language(language, scene).format(money_sum))
                                
                                # ありがとうございました
                                scene = "thanks"
                                print(translate_language(language, scene))
                                print()
                                break
                            elif key == 'n':
                                # 次の商品を指定の位置に置いてください。
                                scene = "next"
                                print(translate_language(language, scene))

NameError: ignored

In [11]:
class main:
    """
    マスタークラス

    """

    def __init__(self, model="YOLO", lang="e"):
        self.model_dict = {'y':"YOLO", 'e':"EFF"}
        self.language_dict = {'k':'Kinyarwanda', 'e':'English', 'f':'French', 'j':'Japanese'}
        self.model = model
        self.lang = lang
        self.check_list = ["y", "n"]

    def mode_select(self):
      """
      初期起動モードを選択させる関数
      標準入力から引数を取って分岐する
      """
      mode_list = ["1", "2", "3"]
      print("\n----------Current status---------")
      print("model : {}\nlanguage : {}".format(self.model, self.language_dict[self.lang]))
      
      print("\n----------Please choice start up mode----------")
      print(" 1 : Start in cash register mode")
      print(" 2 : Start in set up mode")
      print(" 3 : Start in image mode")
      print("--------------------")
      mode = input("Please select and enter the number : ")

      while mode not in mode_list:
          print("\nInvalid number.")
          mode = input("\nPlease enter the correct characters : ")

      if mode == "1":
        pass
      elif mode == "2":
        setup = setup_mode()
        setup.setup_mode_select()

      elif mode == "3":
        pass
        


main().mode_select()



----------Current status---------
model : YOLO
language : English

----------Please choice start up mode----------
 1 : Start in cash register mode
 2 : Start in set up mode
 3 : Start in image mode
--------------------
Please select and enter the number : 2

----------Setting mode----------
Which setting mode?
1 : Model setting
2 : Language setting
3 : Return to the previous
 input : 2

----------Language select mode----------
Kinyarwanda : 'k'
English : 'e'
French : 'f'
Japanese : 'j'
Please select the language : j

言語設定が完了しました : Japanese

言語選択モードを終了しますか？
'y'->終了　 'n'->もう一度選択 : y

----------Setting mode----------
Which setting mode?
1 : Model setting
2 : Language setting
3 : Return to the previous
 input : 1

----------分析モデル選択モード---------- : 
YOLO : 'y'
EfficientNet : 'e'
モデルを選択してください。 : e

モデル設定が完了しました : EFF

モデル選択モードを終了しますか？
'y'->終了　 'n'->もう一度選択 : y

----------Setting mode----------
Which setting mode?
1 : Model setting
2 : Language setting
3 : Return to the previous
 input : 3


TypeError: ignored

In [8]:
from translate import *


class setup_mode(main):
    """
    サブクラス
    設定変更モード

    変更できる設定
    ・分析モデル設定
    ・使用言語設定
    """
    def setup_mode_select(self):
        print("\n----------Setting mode----------")
        print("Which setting mode?")
        stdin = input("1 : Model setting\n2 : Language setting\n3 : Return to the previous\n input : ")
        if stdin == "1":
          self.model_select()
        elif stdin == "2":
          self.language_select()
        if stdin == "3":
          self.mode_select()
        



    def model_select(self):
        print(translate_language(self.lang, "m_select0"))
        print("YOLO : 'y'\nEfficientNet : 'e'")
        stdin = input(translate_language(self.lang, "m_select1"))
        while stdin not in self.model_dict.keys():
            stdin = input(translate_language(self.lang, "select4"))
        
        self.model = self.model_dict[stdin]
        print(translate_language(self.lang, "m_select2").format(self.model))
        exit_flg = input(translate_language(self.lang, "m_select3"))

        while exit_flg not in self.check_list:
            exit_flg = input(translate_language(tmp_lang, "select4"))
        
        if exit_flg == "y":
            self.setup_mode_select()
        else:
            print("")
            self.model_select()
    
    def language_select(self):
        tmp_lang = self.lang

        print(translate_language(self.lang, "l_select0"))
        print("Kinyarwanda : 'k'\nEnglish : 'e'\nFrench : 'f'\nJapanese : 'j'")
        self.lang = input(translate_language(self.lang, "l_select1"))
        
        if self.lang in self.language_dict.keys():
            tmp_lang = self.lang
        while self.lang not in self.language_dict.keys():
            self.lang = input(translate_language(tmp_lang, "select4"))


        print(translate_language(self.lang, "l_select2").format(self.language_dict[self.lang]))
        exit_flg = input(translate_language(self.lang, "l_select3"))

        while exit_flg not in self.check_list:
            exit_flg = input(translate_language(tmp_lang, "select4"))
        
        if exit_flg == "y":
            self.setup_mode_select()
        else:
            print("")
            self.language_select()




#lang = setup_mode()
#lang.setup_mode_select()



In [ ]:
import datetime as dt
import numpy as np
import os
from keras.preprocessing import image
from keras.models import load_model
from keras.preprocessing.image import img_to_array

def register_function():
    # モデルの読み込み
    model_path = model_dict[self.model]
    self_model = load_model(model_path)

    #商品価格等の情報をbottle_masterから読み込み
    name_price_dict = bottle_master.bottle_master_dict()
    _, label_dict = bottle_master.bottle_label_dirpath()
    bottle_label = {}
    for key, value in label_dict.items():
        bottle_label[value] = key
    
    # 撮影するクラスを読み込み
    get_image = shutter2image.shutter()
    
    # 「商品を置いてenterを押してください」
    input(translate_language(self.lang, "scan_start"))
    # 処理時間の計測
    start = time.time()





In [7]:
import logging

logging.warning('Watch out!')  # will print a message to the console
logging.info('I told you so')  # will not print anything

In [ ]:
def shutter(save_path):
    # pi camera 用のライブラリーを使用して、画像を取得
    # 音声再生
    path = os.getcwd()
    sound_path = os.path.join(path, "shutter2image/Camera-Phone03-5.mp3")
    pygame.mixer.init(frequency = 44100)    # 初期設定
    pygame.mixer.music.load(sound_path)
    pygame.mixer.music.play(1)
    sleep(1)
    # 再生の終了
    pygame.mixer.music.stop()
    with picamera.PiCamera() as camera:
        #camera.resolution = (640,480)
        camera.resolution = (300,400)
        camera.start_preview()
        sleep(1.000)
        camera.capture("")